# 数据推理

## 获取训练数据

### 1.dataset.get()

#### 功能

导入原图像和mask，如果需要则随机选择prompt。

#### 数据格式

image: PIL

mask: PIL

point: numpy [N, 2] (x, y)

bbox: numpy [N, 4] (x1, y1, x2, y2)

### 2.dataset.__getitem__()

#### 功能

完成所有预处理操作。在 SAM 中，主要是完成：

1. 导入图像和标签。( PIL -> [H, W, C] )

2. maxResize。这会使得原图像和mask在不改变长宽比例的情况下将最长边resize到符合模型输入的大小，一般是 1024。

3. padding。在上一步的基础上将另一短边padding到模型输入大小。([H, W, C] -> [1024, 1024, C])。

4. normalize 及 toTensor。这不会改变图像大小，只会改变图像属性。([1024, 1024, C] -> [C, 1024, 1024])。

5. mask 会被 clip 到 [0, 1] 之间。

6. prompt 坐标也会随图像大小改变。

#### 数据格式

image: numpy [C, 1024, 1024]

mask: numpy [1, 1024, 1024]

keypoints: numpy [N, 2]

bboxes: numpy [N, 4]

cls_labels: numpy [N]

idx: numpy [1]

### 3.for batch in train_dataloder

#### 功能

遍历 train_dataloader，每次循环随机抽取 batch_size 数据。

#### 数据格式

image: tensor [B, C, 1024, 1024]

mask: tensor [B, 1, 1024, 1024]

keypoints: tensor [B, N, 2]

bboxes: tensor [B, N, 4]

cls_labels: tensor [B, N]

idx: numpy [B, 1]

1. 方法：dataset.get()；


1. 从文件加载为数据。（file -> numpy）[H1, W1, C]
    
2. 图像经过 ResizeLongestSide(sam_model.image_encoder.img_size) 变换，这将改变图像的大小（一般最长边会改变为 1024）。（numpy -> numpy）[H2, W2, C]

3. 图像经过 as_tensor 变化，主要是对通道和数据形式进行改变。（numpy -> tensor）[1, C, H2, W2]

4. 图像经过 Preprocess 变化，这将对图像进行归一化，并且进行 padding（一般图像会成为 1024*1024）。(tensor -> tensor) [1, C, 1024, 1024]

5. 图像通过 dataLoader 进行导入，会进行堆叠形成 batch。(tensor -> tensor) [B, C, 1024, 1024]

5. 图像经过 image_encoder 编码。（tensor -> tensor）[B, 256, 64, 64]

6. 图像和坐标序列 6 所生成的两种 prompt embeddings，一起进入 mask_decoder，生成低分辨率预测和置信度 (在 multimask_output=False 的情况下)。(tensor -> tensor) [B, 1, 256, 256] + [B, 1]

7. 低分辨率预测通过插值，改变大小同序列 4 一致。(tensor -> tensor) [B, 1, 1024, 1024]

8. 高分辨率图像从左上开始取值，改变大小同序列 3 一致。(tensor -> tensor) [B, 1, H2, W2]

9. 数据再通过插值，改变大小同序列 1 一致。(tensor -> tensor) [B, 1, H1, W1]

10. 数据最后再进行判定，如果大于0，则判断为前景，反之为背景。



**附加信息**

1.在序列 6 生成的预测结果中，低分辨率的预测是二分类结果，输出的数据如果大于0，则被认为是前景，反之则为背景；并且输出的置信度处于[0,1]之间。


## point prompt 的一生

1. 生成的坐标格式初始为 [N, 2]，生成的坐标标签初始为 [N, 1]。（坐标的格式为（x,y））[N, 2] + [N, 1]
        
2. 坐标经过 ResizeLongestSide 变换，这会使得坐标数据变换。[N, 2] + [N, 1]

3. 坐标经过 as_tensor 变换，主要是对数据形式和维度进行变换。[1, N, 2] + [1, N]

4. 坐标和标签合并为 1 个 tuple。([1, N, 2], [1, N])

5. 坐标通过 dataLoader 进行集合堆叠形成 batch。([B, N, 2], [B, N])

6. 坐标经过 prompt_encoder，生成稀疏和密集 embeddings。[B, 2, 256] + [B, 256, 64, 64]

## bbox prompt 的一生

1. 生成的坐标格式初始为 [N, 4]。（坐标的格式为（x1,y1,x2,y2））[N, 4]
        
2. 坐标经过 ResizeLongestSide 变换，这会使得坐标数据变换。[N, 4]

3. 坐标经过 as_tensor 变换，主要是对数据形式和维度进行变换。[1, N, 4]

4. 坐标通过 dataLoader 进行集合堆叠形成 batch。([B, N, 4])

4. 坐标经过 prompt_encoder，生成稀疏和密集 embeddings。[B, 2, 256] + [B, 256, 64, 64]